In this notebook, different statistics of the detected ramp planes will be analyzed and visualized.<br>
Also a score will be calculated to determine how well the algorithm works.


In [ ]:
import numpy as np
import pandas as pd
import pcl
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import ros_numpy
import rosbag
from tf.transformations import (
    euler_from_quaternion,
    euler_matrix,
    unit_vector,
    vector_norm,
    quaternion_matrix,
)

pio.templates.default = "plotly_dark"
# %load_ext blackcellmagic


#### **The following values need to be adjusted for each rosbag**


In [ ]:
# Rosbag path
bag_path = "/home/user/rosbags/final/slam/u_c2s_half_odom_hdl.bag"

# ROS topics
odom_topic = "/odom"
lidar_topic = "/velodyne_points"

# Ground truth coordinates of ramp (measured by using globalmap points)
# x_range, y_range = [[32.5, 44], [2, 5.5]]
x_range, y_range = [[20.3, 33], [-0.9, 2.8]]
# Kind of recording: Straight onto curved ramp
# Rosbag path
# bag_path = '/home/user/rosbags/final/slam/u_d2e_hdl.bag'
# bag_path = '/home/user/rosbags/final/slam/straight_wo_ramps_odom_hdl.bag'


Helper functions


In [ ]:
def unpack_bag(bag_path, topic):
    """Extracts all msgs (with time) of a topic from a rosbag"""
    # Load rosbag
    bag = rosbag.Bag(bag_path)

    t_msg = []
    msgs = []
    for topic, msg, t in bag.read_messages(topics=topic):
        if topic == odom_topic:
            msgs.append(msg.pose.pose)
        else:
            msgs.append(msg)
        # Time at which msg was recorded
        t_msg.append(t.to_time())
    return msgs, t_msg


def synchronize_topics(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Synchronizes two topics (even if different rate)."""
    if len(topic1) == len(topic2):
        return topic1, topic2

    # Make sure both topics start around the same time and remove first values of
    # signal which is ahead if this is not the case
    topic1, t_topic1, topic2, t_topic2 = correct_for_time_diff(
        topic1, t_topic1, topic2, t_topic2)

    # Assume first array is bigger
    array_big = t_topic1
    array_small = t_topic2
    is_topic1_bigger = True
    # Swap if not
    if len(t_topic1) < len(t_topic2):
        array_small, array_big = array_big, array_small
        is_topic1_bigger = False
    indices = []

    # For each time of the small array, check which is
    # the closest msg in the big array
    for i, v in enumerate(array_small):
        # Offset big array with a msg from small array
        diff = np.abs(array_big - v)
        # Index where time difference is the smallest
        idx = diff.argmin()
        if idx < len(array_small) and diff[idx] < t_thresh:
            indices.append(idx)

    print(
        "t_start_diff: {:.3f} s, t_end_diff: {:.3f} s".format(
            np.abs(array_small[0] - array_big[indices][0]),
            np.abs(array_big[indices][-1] -
                   array_small[len(array_big[indices] - 1)]),
        )
    )

    if is_topic1_bigger:
        topic1_synched = topic1[indices]
        topic2_synched = topic2[: len(topic1_synched)]
        return topic1_synched, topic2_synched
    else:
        topic2_synched = topic2[indices]
        topic1_synched = topic1[: len(topic2_synched)]
        return topic1_synched, topic2_synched


def correct_for_time_diff(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Checks for a time offset of two topics at the beginning and shorts ahead signal"""
    # Assume first topic is behind and convert to numpy array
    topic_behind, topic_ahead = np.asarray(topic1), np.asarray(topic2)
    t_topic_behind, t_topic_ahead = np.asarray(t_topic1), np.asarray(t_topic2)
    is_topic1_behind = True
    # Swap if other way around
    if t_topic2[0] - t_topic1[0] > t_thresh:
        topic_ahead, topic_behind = topic_behind, topic_ahead
        t_topic_ahead, t_topic_behind = t_topic_behind, t_topic_ahead
        is_topic1_behind = False
    # No offset
    elif np.abs(t_topic1[0] - t_topic2[0]) < t_thresh:
        return tuple([np.asarray(x) for x in [topic1, t_topic1, topic2, t_topic2]])

    # Search for time of ahead topic which is closest to start of behind topic
    diff = np.abs(t_topic_ahead - t_topic_behind[0])
    # Index where time difference is the smallest
    idx = diff.argmin()
    # Remove first couple values to make both start at the same time
    topic_ahead = topic_ahead[idx:]
    t_topic_ahead = t_topic_ahead[idx:]

    if is_topic1_behind:
        print("topic1 was behind by {}".format(idx))
        return topic_behind, t_topic_behind, topic_ahead, t_topic_ahead
    else:
        print("topic2 was behind by {}".format(idx))
        return topic_ahead, t_topic_ahead, topic_behind, t_topic_behind


Load data (pointcloud and odom) from rosbag


In [ ]:
# Odometer from hdl_slam
pose, t_pose = unpack_bag(bag_path, odom_topic)
# Velodyne point cloud
lidar, t_lidar = unpack_bag(bag_path, lidar_topic)

# Synchronize both topics (odom usually is behind lidar)
pose, lidar = synchronize_topics(pose, t_pose, lidar, t_lidar)


Class of the actual ramp detection algorithm


In [ ]:
# My ROS Node
class VisualDetection():

    def __init__(self):
        self.rp = [0, 0]
        # x coordinates where ramp starts
        self.ramp_start = x_range[0]

    def spin(self, cloud, pose):
        self.cloud = cloud
        self.pose = pose
        # Robosense Lidar has a rate of 10 Hz
        # Convert PointCloud2 msg to numpy array
        pc_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(
            self.cloud, remove_nans=True)

        # Apply lidar to car frame transformation
        # Use tf average of all bags instead of calculating new transform
        pc_array_tf = self.transform_pc(pc_array, rpy=(0, 0.0083, 0),
                                        translation_xyz=(-2.36, -0.05, 1.8753))

        # Filter unwanted points (to reduce point cloud size) with passthrough filter
        pc_array_cut = self.reduce_pc(pc_array_tf, (0, 30), (-2, 2), (-1, 2))

        # Convert numpy array to pcl object
        pc_cut = self.array_to_pcl(pc_array_cut)

        # Downsample point cloud using voxel filter to further decrease size
        pc_small = self.voxel_filter(pc_cut, 0.1)

        # Perform RANSAC until no new planes are being detected
        plane_coor, data = self.plane_detection(pc_small, 20, 4)
        return plane_coor, data, self.relative_to_absolute(pc_array_cut)

    @staticmethod
    def array_to_pcl(pc_array):
        """Get pcl point cloud from numpy array"""
        pc = pcl.PointCloud()
        pc.from_array(pc_array.astype('float32'))
        return pc

    @staticmethod
    def quat_from_vectors(vec1, vec2):
        """Quaternion that aligns vec1 to vec2"""
        # Normalize vectors
        a, b = unit_vector(vec1), unit_vector(vec2)
        c = np.cross(a, b)
        d = np.dot(a, b)

        # Rotation axis
        axis = c / vector_norm(c)
        # Rotation angle (rad)
        ang = np.arctan2(vector_norm(c), d)

        # Quaternion ([x,y,z,w])
        quat = np.append(axis*np.sin(ang/2), np.cos(ang/2))
        return quat

    @staticmethod
    def transform_pc(pc, rpy=(0, 0, 0), translation_xyz=(1.7, 0, 1.7)):
        """Transformation from Lidar frame to car frame. Rotation in rad and translation in m."""
        # Extract euler angles
        roll, pitch, yaw = rpy
        # Extract translations
        transl_x, transl_y, transl_z = translation_xyz

        # Rotation matrix
        rot = euler_matrix(roll, pitch, yaw, 'sxyz')[:3, :3]
        # Apply rotation
        pc_tf = np.inner(pc, rot)

        # Translation to front of the car
        translation_to_front = [transl_x, transl_y, transl_z]
        # Combine rotation and translation
        pc_tf += translation_to_front
        return pc_tf

    @staticmethod
    def reduce_pc(pc, x_range, y_range, z_range):
        """Removes points outside of box"""
        # Filter array
        pc_cut = pc[
            (pc[:, 0] > x_range[0])
            & (pc[:, 0] < x_range[1])
            & (pc[:, 1] > y_range[0])
            & (pc[:, 1] < y_range[1])
            & (pc[:, 2] > z_range[0])
            & (pc[:, 2] < z_range[1])
        ]
        return pc_cut

    @staticmethod
    def voxel_filter(pc, leaf_size):
        """Downsample point cloud using voxel filter"""
        vgf = pc.make_voxel_grid_filter()
        # Leaf_size is the length of the side of the voxel cube in m
        vgf.set_leaf_size(leaf_size, leaf_size, leaf_size)
        pc_filtered = vgf.filter()
        return pc_filtered

    def plane_detection(self, pc, min_points, max_planes):
        """Detects all planes in point cloud"""
        # Count number of iterations
        counter = 0
        # Standard values for ramp angle and distance if no detection
        ramp_stats = ([], [])
        # Detect planes until ramp found or conditions not met anymore
        while pc.size > min_points and counter < max_planes:
            # Detect most dominate plane and get inliers and normal vector
            inliers_idx, coefficients = self.ransac(pc)
            # Normal vector of plane
            n_vec = coefficients[:-1]

            # Split pointcloud in outliers of plane and inliers
            plane = pc.extract(inliers_idx)
            pc = pc.extract(inliers_idx, negative=True)

            # Exit if plane is empty (RANSAC did not find anything)
            if plane.size == 0:
                return ramp_stats

            # Ignore planes parallel to the side or front walls
            if self.is_plane_near_ground(n_vec):
                # Check if ramp conditions are fullfilled
                is_ramp, data = self.ramp_detection(
                    plane, n_vec, (3, 9), (2, 6))
                # Ramp conditions met
                if is_ramp:
                    plane_global = self.relative_to_absolute(plane)
                    return (plane_global, data)
            counter += 1
        return ramp_stats

    def relative_to_absolute(self, pc):
        """Transforms relative lidar data to absolute by adding translation rotating"""
        # pc_arr = pc.to_array()
        pc_arr = np.array(pc)

        # Odometer
        pos = self.pose.position
        translation = [pos.x, pos.y, pos.z]
        ori = self.pose.orientation
        quat = [ori.x, ori.y, ori.z, ori.w]
        roll, pitch, yaw = euler_from_quaternion(quat)

        # Rotation matrix
        rot = quaternion_matrix(quat)[:3, :3]
        # Apply rotation
        pc_tf = np.inner(pc_arr, rot)

        # Combine rotation and translation
        pc_tf += translation
        return pc_tf

    def ramp_detection(self, plane, n_vec, angle_range, width_range):
        """Checks if conditions to be considered a ramp are fullfilled."""
        # Convert pcl plane to numpy array
        plane_array = plane.to_array()

        # Calculate angle [deg] between normal vector of plane and ground
        angle = self.angle_calc([0, 0, 1], n_vec)
        # Get ramp width (difference between y-values)
        width = max(plane_array[:, 1]) - min(plane_array[:, 1])
        # Ramp distance (average x-value of nearest points of the plane)
        n_nearest = 10
        dist = np.mean(np.sort(plane_array[:n_nearest, 0]))

        true_dist = self.ramp_start - self.pose.position.x

        # Assert ramp angle and width thresholds
        if (angle_range[0] <= angle <= angle_range[1]
                and width_range[0] <= width <= width_range[1]):
            return True, [angle, width, dist, true_dist]
        return False, [angle, width, dist, true_dist]

    def ransac(self, pc):
        """Finds inliers and normal vector of dominant plane"""
        # 50?
        seg = pc.make_segmenter_normals(50)
        # Doubles the speed if True
        seg.set_optimize_coefficients(True)
        seg.set_model_type(pcl.SACMODEL_NORMAL_PLANE)
        seg.set_method_type(pcl.SAC_RANSAC)
        # How close a point must be to model to be considered inlier
        seg.set_distance_threshold(0.11)
        # normal_distance_weight?
        seg.set_normal_distance_weight(0.01)
        # How many tries
        seg.set_max_iterations(100)
        indices, coefficients = seg.segment()
        return indices, coefficients

    def split_pc(self, pc, inliers):
        """Extract detected plane from point cloud and split into two pcs"""
        # Get point cooridnates of plane
        detected_plane = [pc[i] for i in inliers]
        # Point cloud of detected plane (inliers)
        pc_inliers = pc.extract(inliers)

        # Point cloud of outliers
        outlier_indices = list(
            set(np.arange(pc.size)).symmetric_difference(inliers))
        pc_outliers = pc.extract(outlier_indices)

        return pc_outliers, detected_plane, pc_inliers

    def is_plane_near_ground(self, v, threshold=0.8):
        """Returns True if plane is on the ground (and false if e.g. side wall)"""
        return abs(v[2]) > threshold

    def angle_calc(self, v1, v2, degrees=True):
        """Calculate angle between two vectors (planes)"""
        # Assuming both vectors can be rotated alongside one axis to be aligned
        dot = np.dot(v1, v2)
        if dot <= 1:
            angle = np.arccos(dot)
        else:
            angle = 0

        if degrees is True:
            return np.degrees(angle)
        else:
            return angle


Run algorithm with rosbag data.<br>
Store many different calculated metrics generated by the algorithm in a pandas data frame (to allow a faster plotting).


In [ ]:
# Create instance of class (using standard parameters):
vd = VisualDetection()
# Lists to fill, will contain entry for each plane
planes = []
ramp_stats = []
true_dist = []
all_points = []
for i in range(len(lidar)):
    plane_points, data, pc_whole = vd.spin(lidar[i], pose[i])
    planes.append(plane_points)
    ramp_stats.append(data)
    all_points.append(pc_whole)
    # True distance to start of ramp
    true_dist.append(x_range[0] - pose[i].position.x)
true_dist = np.array(true_dist)

# Remove empty lists (when no ramp has been detected)
# because information such as angle, width of ramp etc
# are only stored when a ramp is detected, 0 otherwise
ramp_arrays = [x for x in planes if not isinstance(x, list)]
ramp_stats = [x for x in ramp_stats if x != []]
# Get indices where ramp has been detected
ramp_indices = [i for i, v in enumerate(planes) if not isinstance(v, list)]

# Convert list to dictionary
dic = []
for i, arr in enumerate(ramp_arrays):
    for j, point in enumerate(arr):
        dic.append(
            {"sampleIdx": i, "pointIdx": j,
                "x": point[0], "y": point[1], "z": point[2]}
        )
# And finally to pandas data frame
df = pd.DataFrame(dic)
# Reorder columns
df = df[["sampleIdx", "pointIdx", "x", "y", "z"]]

# Add some buffer to the true ramp region (for tidy plots)


def add_buffer(range, margin=3):
    for i, v in enumerate(range):
        if i == 0:
            low = int(round(v)) - margin
        else:
            high = int(round(v)) + margin
    return [low, high]


x_fixed = add_buffer(x_range)
y_fixed = add_buffer(y_range)

# Check if a point lies within ramp region
lies_inside = []
for i, x in enumerate(df["x"]):
    if x_range[0] < x < x_range[1]:
        if y_range[0] < df["y"][i] < y_range[1]:
            # True if x and y coordinate inside region
            lies_inside.append(True)
        else:
            lies_inside.append(False)
    else:
        lies_inside.append(False)
# Add column (bool: if point lies in region) to data frame
df["inlier"] = lies_inside

# Calculate how many points of each sample lie in ramp region
true_inliers = []
samples_num = df.sampleIdx.max() + 1
for i in range(samples_num):
    # Bool list of inliers and outliers of sample
    bool_lst = df[df["sampleIdx"] == i]["inlier"]
    # Percentage of inliers of sample
    true_inliers.append(sum(bool_lst) / float(len(bool_lst)))

# New dataframe with stats for each frame
# Structure reminder of ramp_stats: [angle, width, dist, true_dist]
dic = []
for i in range(samples_num):
    dic.append(
        {
            "sampleIdx": i,
            "TrueInliers": true_inliers[i],
            "Angle": ramp_stats[i][0],
            "Width": ramp_stats[i][1],
            "Dist": ramp_stats[i][2],
            "TrueDist": ramp_stats[i][3],
        }
    )
# Convert dictionary to dataframe
df_stats = pd.DataFrame(dic)
# Reorder columns
df_stats = df_stats[["sampleIdx", "TrueInliers",
                     "Angle", "Width", "Dist", "TrueDist"]]

# Filter the best detections
df_stats_good = df_stats[df_stats["TrueInliers"] > 0.7]
# Filter the bad detections
df_stats_bad = df_stats[df_stats["TrueInliers"] < 0.3]
# Filter the average detections
df_stats_avg = df_stats[0.3 < df_stats["TrueInliers"]]
df_stats_avg = df_stats_avg[df_stats_avg["TrueInliers"] < 0.7]


## Score calculation

To calculate the score, a range has to be specified in which the lidar should be able to detect the ramp. Looking at the histogram, the range between 4 and 10 m before the ramp seems reasonable.

Now filter all the data, such that only samples in the range of 4 to 10 m before the ramp are used.


In [ ]:
def calc_score(min_dist, max_dist):
    # Sensitivity
    # How many sample were recorded in given range
    expected_detections = len(
        true_dist[(min_dist < true_dist) & (true_dist < max_dist)]
    )
    # How many samples were identified as ramp in given range
    actual_detections = len(
        df_stats["TrueDist"][
            (min_dist < df_stats["TrueDist"]) & (
                df_stats["TrueDist"] < max_dist)
        ]
    )
    # Calculate ratio
    sensitivity = float(actual_detections) / expected_detections
    print("Out of {} recorded samples {} were detected in the range of {} to {}m before the ramp.".format(
        expected_detections, actual_detections, min_dist, max_dist))
    print("Resulting in a score of {:.2f}%\n".format(sensitivity * 100))

    # Inlier score
    f = df_stats["TrueInliers"][
        (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)
    ]
    try:
        score2 = sum(f) / len(f)
    except ZeroDivisionError:
        score2 = 0
    print("Of the {} detected planes {:.2f}% inlier points were actually inside the ramp region.\n".format(
        actual_detections, score2 * 100))

    # False positives?
    f = df_stats["TrueInliers"][
        (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)
    ]
    thresh = 0.5
    true_detections = sum(f > thresh)
    try:
        score3 = float(true_detections) / len(f)
    except ZeroDivisionError:
        score3 = 0
    print("Of the {} detected planes {} had at least {}% of points inside the ramp region.".format(
        actual_detections, true_detections, int(thresh * 100)))
    print("Resulting in a score of {:.2f}%\n\n".format(score3 * 100))


calc_score(4, 10)
calc_score(0, 100)


# Plots


**How was this plot created:**

- Calculate distance to start of ramp using odometry data from hdl_slam
- Calculate how many lidar samples were recorded during a 1m drive (e.g. car took 0.5s for 1m $\rightarrow$ lidar took 5 samples (because $f_\text{Lidar}=$ 10Hz))
- Calculate how many samples of them were actually identified as a ramp by the algorithm
- Represent both values in a bar plot
- Value at bar represents the lower range (e.g. bar at 15 means 16m to 15m)

**What to take from this plot:**<br>

- At which distance to the ramp is the lidar most reliable?<br>
- $\rightarrow$ Range between 10m to 4m seems to be the most reliable


In [ ]:
expected_detections = []
actual_detections = []
# Split dataset in 1m intervals
for i in range(15, -5, -1):
    # Filter distance to ramp (from odom data) in 1m intervals
    expected_detections.append(
        len(true_dist[(i < true_dist) & (true_dist < i + 1)]))
    # Get corresponding distance estimations
    actual_detections.append(
        len(
            df_stats["TrueDist"][
                (i < df_stats["TrueDist"]) & (df_stats["TrueDist"] < i + 1)
            ]
        )
    )
# Create pandas data frame
eval_df = pd.DataFrame()
# Distance to ramp e.g. a value of 15 means interval from 16m to 15m
eval_df["distToRamp"] = range(15, -5, -1)
eval_df["expectedDetections"] = expected_detections
eval_df["actualDetections"] = actual_detections

# Plot
fig = go.Figure()
fig.add_trace(
    go.Bar(
        y=eval_df["expectedDetections"],
        x=eval_df["distToRamp"],
        name="False negatives (Not detected)",
        offsetgroup=0,
    )
)
fig.add_trace(
    go.Bar(
        y=eval_df["actualDetections"],
        x=eval_df["distToRamp"],
        name="True positives (Detected)",
        offsetgroup=0,
    )
)
fig.update_xaxes(title_text="Distance to ramp [m]", autorange="reversed")
fig.update_yaxes(title_text="Number of ramps detected")
fig.update_layout(
    title_text="How many samples were collected in 1m intervals and how many of them have been identified as ramp")
fig.show()


**How was this plot created:**

- Every time a ramp is detected certain metrics such as angle, width, distance are estimated by the algorithm
- Compare predictions to the true values, true values were measured by hand
- Calculate some stats like standard deviation or average error and add them
- Represent values as a line plot

**What to take from this plot:**

- (Lars wanted it)
- Distance to ramp:
  - Distance is estimated quite well up to about 3m before the ramp
  - There does seem to be a slight offset (manual measurement might have been wrong)
- Angle:
  - Quite a bit of variance at far and close distance, but very small deviation in the range 9m to 3m
  - Because ramp does not have a constant angle, angle estimation of 4° etc. at the beginning of the ramp might be correct
- Width:
  - Mhm...


In [ ]:
# Distance estimation evaluation
diff = df_stats["Dist"] - df_stats["TrueDist"]
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df_stats["TrueDist"],
               y=df_stats["TrueDist"], name="Measured (hdl)")
)
fig.add_trace(go.Scatter(
    x=df_stats["TrueDist"], y=df_stats["Dist"], name="Estimated"))
fig.update_traces(mode="lines+markers")
fig.update_layout(
    title="Difference between estimated and actual distance to ramp")
fig.add_annotation(
    text="Standard deviation: {:.2f}m<br>Average error: {:.2f}m<br>Median error: {:.2f}m".format(
        np.std(diff), np.mean(diff), np.median(diff)
    ),
    xref="paper",
    yref="paper",
    x=1,
    y=1,
    showarrow=False,
)
fig.update_xaxes(title_text="(actual) Distance [m]", autorange="reversed")
fig.update_yaxes(title_text="Distance [m]")
fig.show()

# Angle estimation evaluation
true_angle = 6
diff = df_stats["Angle"] - true_angle
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_stats["TrueDist"],
        y=true_angle * np.ones(len(df_stats)),
        name="Measured",
        mode="lines",
    )
)
fig.add_trace(
    go.Scatter(
        x=df_stats["TrueDist"],
        y=df_stats["Angle"],
        name="Estimated",
        mode="lines+markers",
    )
)
fig.update_layout(
    title="Difference between estimated and actual angle of ramp")
fig.add_annotation(
    text="Average: {:.2f}<br>Standard deviation: {:.2f}°<br>Average error: {:.2f}°<br>Median error: {:.2f}°".format(
        np.mean(df_stats["Angle"]), np.std(
            diff), np.mean(diff), np.median(diff)
    ),
    xref="paper",
    yref="paper",
    x=1,
    y=1,
    showarrow=False,
)
fig.update_xaxes(title_text="Distance to ramp [m]", autorange="reversed")
fig.update_yaxes(title_text="Ramp angle [°]")
fig.show()

# Width estimation evaluation
# Whole width (including curb)
true_width = y_range[-1] - y_range[0]
# Only street (measured by hand)
true_width_drive = 2.9
diff = df_stats["Width"] - true_width
diff_drive = df_stats["Width"] - true_width_drive
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_stats["TrueDist"],
        y=true_width * np.ones(len(df_stats)),
        name="Measured (whole)",
        mode="lines",
    )
)
fig.add_trace(
    go.Scatter(
        x=df_stats["TrueDist"],
        y=true_width_drive * np.ones(len(df_stats)),
        name="Measured (drivable)",
        mode="lines",
    )
)
fig.add_trace(
    go.Scatter(
        x=df_stats["TrueDist"],
        y=df_stats["Width"],
        name="Estimated",
        mode="lines+markers",
    )
)
fig.update_layout(
    title="Difference between estimated and actual width of ramp")
fig.add_annotation(
    text="Average: {:.2f}<br>Standard deviation: {:.2f}m<br>Average error: {:.2f}m<br>Median error: {:.2f}m".format(
        np.mean(df_stats["Width"]), np.std(
            diff), np.mean(diff), np.median(diff)
    ),
    xref="paper",
    yref="paper",
    x=1,
    y=1,
    showarrow=False,
)
fig.add_annotation(
    text="Standard deviation: {:.2f}m<br>Average error: {:.2f}m<br>Median error: {:.2f}m".format(
        np.std(diff_drive), np.mean(diff_drive), np.median(diff_drive)
    ),
    xref="paper",
    yref="paper",
    x=1,
    y=0,
    showarrow=False,
)
fig.update_xaxes(title_text="Distance to ramp [m]", autorange="reversed")
fig.update_yaxes(title_text="Ramp width [m]")
fig.show()


**How was this plot created:**

- Every time the algorithm detects a ramp, the coordinates of the inliers are stored
- But often times not all points in ramp region are detected
- Display points detected by algorithm in one color and all the other points recorded by the lidar in another color
- Add an area of the true ramp region (measured by hand)
- Calculate percentage of inliers how lie in ramp region
- Do this for every sample which was detected as ramp
- Add a slider (just to show off)

**What to take from this plot:**

- (Plotly is cool)
- Only very few "lines" are thrown on the ground / on to the ramp
- Distances of over 2m between two lines are common (especially at higher distances)


In [ ]:
fig = go.Figure()
trace_list1 = []
trace_list2 = []
# Add traces, one for each slider step
for step in range(df["sampleIdx"].max() + 1):
    trace_list1.append(
        go.Scatter(
            visible=False,
            mode="markers",
            x=df[df["sampleIdx"] == step]["x"],
            y=df[df["sampleIdx"] == step]["y"],
            name="Detected by algorithm as ramp",
        )
    )
    trace_list2.append(
        go.Scatter(
            visible=False,
            mode="markers",
            marker=dict(opacity=0.5),
            x=all_points[ramp_indices[step]][:, 0],
            y=all_points[ramp_indices[step]][:, 1],
            name="All lidar points",
        )
    )
fig = go.Figure(data=trace_list2 + trace_list1)
fig.data[0].visible = True
fig.add_shape(
    type="rect",
    x0=x_range[0],
    x1=x_range[1],
    y0=y_range[0],
    y1=y_range[1],
    fillcolor="red",
    opacity=0.2,
)
# Set static axes limits
fig.update_xaxes(range=x_fixed)
fig.update_yaxes(range=y_fixed)

# Create and add slider
steps = []
for i in range(len(fig.data) / 2):
    step = dict(
        method="update",
        args=[
            {"visible": [False] * len(fig.data)},
            {
                "title": "{:.2f}% of detected points lie in the ramp region. Was detected {:.2f}m infront of ramp".format(
                    true_inliers[i] * 100, df_stats.iloc[i, -1]
                )
            },
        ],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][
        i + len(fig.data) / 2
    ] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(active=0, currentvalue={
                "prefix": "Deteced plane: "}, steps=steps)]
fig.update_layout(
    sliders=sliders,
    xaxis_title="Global x coor [m]",
    yaxis_title="Global y coor [x]",
    showlegend=True,
)
fig.show()


**How was this plot created:**

- Similar to previous plot
- For every recorded lidar sample all points are displayed
- Add an area of the true ramp region (measured by hand)

**What to take from this plot:**

- Just gives an idea of what the lidar sees, might be useful to optimize mount pitch angle
- Also might give information, about why some samples are not recognized as ramp, eventhough they should be


In [ ]:
fig = go.Figure()
# Add traces, one for each slider step
for step in range(len(all_points)):
    fig.add_trace(
        go.Scatter(
            visible=False,
            mode="markers",
            x=all_points[step][:, 0],
            y=all_points[step][:, 1],
            name="All lidar points",
        )
    )
fig.data[0].visible = True
fig.add_shape(
    type="rect",
    x0=x_range[0],
    x1=x_range[1],
    y0=y_range[0],
    y1=y_range[1],
    fillcolor="red",
    opacity=0.2,
)
# Set static axes limits
fig.update_xaxes(range=x_fixed)
fig.update_yaxes(range=y_fixed)

# Create and add slider
steps = []
for i in range(len(fig.data)):
    ramp_det = "A" if i in ramp_indices else "No"
    step = dict(
        method="update",
        args=[
            {"visible": [False] * len(fig.data)},
            {
                "title": "{:.2f}m infront of ramp. {} ramp has been detected".format(
                    #   true_inliers[i]*100, df_stats.iloc[i,-1])}],  # layout attribute
                    true_dist[i],
                    ramp_det,
                )
            },
        ],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(active=0, currentvalue={
                "prefix": "Deteced plane: "}, steps=steps)]
fig.update_layout(
    sliders=sliders,
    xaxis_title="Global x coor [m]",
    yaxis_title="Global y coor [x]",
    showlegend=True,
)
fig.show()


**How was this plot created:**

- Every time the algorithm detects a ramp, the coordinates of the inlier points are saved
- Put all point coordinates of every detection together
- Add an area of the true ramp region (measured by hand)
- Display as a scatter plot and also as a heatmap

**What to take from this plot:**

- Shows which points are most commonly detected
- Majority of points are inside ramp region
- ?


In [ ]:
# All points
fig = px.scatter(x=df["x"], y=df["y"], title="Coordinates of all points")
fig.add_shape(
    type="rect",
    x0=x_range[0],
    x1=x_range[1],
    y0=y_range[0],
    y1=y_range[1],
    fillcolor="red",
    opacity=0.2,
)
fig.show()

fig = px.density_heatmap(
    df,
    x="x",
    y="y",
    nbinsx=200,
    nbinsy=50,
    title="Heatmap, where do the most points get detected?",
)
fig.add_shape(
    type="rect",
    x0=x_range[0],
    x1=x_range[1],
    y0=y_range[0],
    y1=y_range[1],
    line=dict(color="White", width=3),
    opacity=0.9,
)
fig.show()


**How was this plot created:**

- Every time a ramp is detected certain metrics such as angle, width, distance are estimated by the algorithm
- Because the inliers of a detected ramp do not always lie in the ramp region, split the dataset in "good" and "bad" sets
- "good" sets are samples, where more than 70% of inliers lie in the ramp region, whereas less than 30% do so in a "bad" sample
- For each set display the metrics as a histogram

**What to take from this plot:**

- Helps to selected a good metric, which can be used as an constraint in the detection
- Look for metrics with a low variance in the "good" dataset $\rightarrow$ potentially good metric
- Look for metrics with a high variance in the "good" dataset $\rightarrow$ potentially bad metric
- Look for metrics with a lwo variance in the "bad" dataset $\rightarrow$ could be used as an inverted condition
- Angle and width seem to be fairly good metrics (condition ranges of both metrics were already reduced, was more obvious before)


In [ ]:
# Stats for good detection
fig = make_subplots(rows=2, cols=2)
fig.add_trace(
    go.Histogram(x=df_stats_good["Angle"], nbinsx=10, name="Angle"), row=1, col=1
)
fig.add_trace(
    go.Histogram(x=df_stats_good["Dist"], nbinsx=10, name="Distance"), row=1, col=2
)
fig.add_trace(
    go.Histogram(x=df_stats_good["Width"], nbinsx=10, name="Width"), row=2, col=1
)
fig.add_trace(
    go.Histogram(x=df_stats_good["TrueInliers"],
                 nbinsx=20, name="True Inliers"),
    row=2,
    col=2,
)
fig.update_layout(
    title="Couple of stats of well detected ramp planes, which lie > 70% in the desired area"
)
fig.show()

# Stats for bad detection
fig = make_subplots(rows=2, cols=2)
fig.add_trace(
    go.Histogram(x=df_stats_bad["Angle"], nbinsx=10, name="Angle"), row=1, col=1
)
fig.add_trace(
    go.Histogram(x=df_stats_bad["Dist"], nbinsx=10, name="Distance"), row=1, col=2
)
fig.add_trace(
    go.Histogram(x=df_stats_bad["Width"], nbinsx=10, name="Width"), row=2, col=1
)
fig.add_trace(
    go.Histogram(x=df_stats_bad["TrueInliers"],
                 nbinsx=20, name="True Inliers"),
    row=2,
    col=2,
)
fig.update_layout(
    title="Couple of stats of badly detected ramp planes, which lie < 30% in the desired area"
)
fig.show()
